In [4]:
#! pip install --upgrade --quiet bitsandbytes datasets peft transformers trl
! pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121


In [20]:
import os
import torch
import bitsandbytes as bnb
from pathlib import Path

DATA_PATH = Path("../data")
OMI_PATH_processed = DATA_PATH / "processed" / "omi-health"
OMI_PATH_raw = DATA_PATH / "raw" / "omi-health"

print(f"PyTorch CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device count: {torch.cuda.device_count()}")
    print(f"Current CUDA device name: {torch.cuda.get_device_name(0)}")

print(f"Bitsandbytes version: {bnb.__version__}")
# You could also try a simple operation with bnb if you have a model loaded

PyTorch CUDA available: True
CUDA device count: 1
Current CUDA device name: NVIDIA RTX A4000
Bitsandbytes version: 0.45.5


In [21]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "google/txgemma-9b-chat"

# Use 4-bit quantization to reduce memory usage
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    device_map={"":0},
    torch_dtype="auto",
    attn_implementation="eager",
)

Loading checkpoint shards: 100%|██████████| 4/4 [00:12<00:00,  3.10s/it]


In [12]:
import pandas as pd

# Load the dataset
train_df = pd.read_csv(OMI_PATH_processed / "train.csv")
train_df.head()

,dialogue,subjective,objective,assessment,plan,event_tags
0,"Doctor: Hello, how can I help you today?\nPatient: My son has been having some issues with speech and development. He's 13 years old now.\nDoctor: I see. Can you tell me more about his symptoms? Does he have any issues with muscle tone or hypotonia?\nPatient: No, he doesn't have hypotonia. But he has mild to moderate speech and developmental delay, and he's been diagnosed with attention deficit disorder.\nDoctor: Thank you for sharing that information. We'll run some tests, including an MRI, to get a better understanding of your son's condition. \n(After the tests)\nDoctor: The MRI results are in, and I'm glad to say that there are no structural brain anomalies. However, I did notice some physical characteristics. Does your son have any facial features like retrognathia, mild hypertelorism, or a slightly elongated philtrum and thin upper lip?\nPatient: Yes, he has all of those features. His hands are also broad and short. And his feet have mild syndactyly of the second and third toe, with a sandal gap in both feet.\nDoctor: Thank you for confirming that. We also conducted Whole Exome Sequencing (WES) analyses, and we found a de novo frameshift variant in his genetic makeup. Specifically, it's Chr1(GRCh37):g.244217335del, NM_205768.2(ZBTB18):c.259del(p.(Leu87Cysfs*21)). This leads to a premature termination codon located more than 400 codons upstream of the canonical termination codon.\nPatient: What does that mean for my son?\nDoctor: This genetic variant may be contributing to your son's speech, developmental delay, and attention deficit disorder. It's important that we continue monitoring his progress and provide appropriate support for his development.\nPatient: What should we do for follow-up?\nDoctor: Regular visits with a speech and language therapist, an occupational therapist, and a psychologist can help address your son's developmental and attention deficit disorder needs. I will also recommend regular check-ups with me to monitor his growth and overall health.\nPatient: Thank you, doctor. We will follow your recommendations and keep an eye on his progress.","The patient's mother reports that her 13-year-old son has mild to moderate speech and developmental delays and has been diagnosed with attention deficit disorder. She denies any issues with muscle tone or hypotonia. The patient also exhibits certain physical characteristics, including retrognathia, mild hypertelorism, an elongated philtrum, thin upper lip, broad and short hands, mild syndactyly of the second and third toes, and a sandal gap in both feet.","An MRI of the brain showed no structural anomalies. Whole Exome Sequencing (WES) revealed a de novo frameshift variant Chr1(GRCh37):g.244217335del, NM_205768.2(ZBTB18):c.259del(p.(Leu87Cysfs*21)), indicating a premature termination codon located more than 400 codons upstream of the canonical termination codon.","The primary diagnosis is a genetic disorder associated with the identified frameshift mutation, which likely contributes to the patient's speech and developmental delays and attention deficit disorder. The physical characteristics and genetic findings suggest a specific syndrome, which needs further correlation with clinical findings and genetic counseling.","The management plan includes regular follow-up visits with a speech and language therapist, an occupational therapist, and a psychologist to support the patient's developmental needs and address his attention deficit disorder. Regular medical check-ups will monitor his growth and overall health. Genetic counseling for the family is also recommended to discuss the implications of the genetic findings and potential familial inheritance.",['(After the tests)']
1,"Doctor: Hello, what brings you in today?\nPatient: Hi, my 21-month-old son has been experiencing weakness in his lower extremities and lumbar pain after a mild upper respiratory tract infection.\nDoctor: I see. Did you consult any m

In [13]:
train_df.iloc[0,0]

"Doctor: Hello, how can I help you today?\nPatient: My son has been having some issues with speech and development. He's 13 years old now.\nDoctor: I see. Can you tell me more about his symptoms? Does he have any issues with muscle tone or hypotonia?\nPatient: No, he doesn't have hypotonia. But he has mild to moderate speech and developmental delay, and he's been diagnosed with attention deficit disorder.\nDoctor: Thank you for sharing that information. We'll run some tests, including an MRI, to get a better understanding of your son's condition. \n(After the tests)\nDoctor: The MRI results are in, and I'm glad to say that there are no structural brain anomalies. However, I did notice some physical characteristics. Does your son have any facial features like retrognathia, mild hypertelorism, or a slightly elongated philtrum and thin upper lip?\nPatient: Yes, he has all of those features. His hands are also broad and short. And his feet have mild syndactyly of the second and third toe, 

Trying out the pretrained model

In [45]:
prompt = ("Create a Medical SOAP note summary from the dialogue, following these guidelines:\n"
          "S (Subjective): Summarize the patient's reported symptoms, including chief complaint and relevant history. Rely on the patient's statements as the primary source and ensure standardized terminology.\n    "
          "O (Objective): Highlight critical findings such as vital signs, lab results, and imaging, emphasizing important details like the side of the body affected and specific dosages. Include normal ranges where relevant.\n    "
          "A (Assessment): Offer a concise assessment combining subjective and objective data. State the primary diagnosis and any differential diagnoses, noting potential complications and the prognostic outlook.\n    "
          "P (Plan): Outline the management plan, covering medication, diet, consultations, and education. Ensure to mention necessary referrals to other specialties and address compliance challenges.\n    "

          "Considerations: Compile the report based solely on the transcript provided. Maintain confidentiality and document sensitively. Use concise medical jargon and abbreviations for effective doctor communication. Add explanations to medical terms if needed\n    Format the summary in a clean, simple list format without using markdown or bullet points. Use 'S:', 'O:', 'A:', 'P:' directly followed by the text. Avoid any styling or special characters.")

full_prompt = f"{prompt}\n\nDialogue to summarize:\n{train_df.iloc[0,0]}"
print(full_prompt)

Create a Medical SOAP note summary from the dialogue, following these guidelines:
S (Subjective): Summarize the patient's reported symptoms, including chief complaint and relevant history. Rely on the patient's statements as the primary source and ensure standardized terminology.
    O (Objective): Highlight critical findings such as vital signs, lab results, and imaging, emphasizing important details like the side of the body affected and specific dosages. Include normal ranges where relevant.
    A (Assessment): Offer a concise assessment combining subjective and objective data. State the primary diagnosis and any differential diagnoses, noting potential complications and the prognostic outlook.
    P (Plan): Outline the management plan, covering medication, diet, consultations, and education. Ensure to mention necessary referrals to other specialties and address compliance challenges.
    Considerations: Compile the report based solely on the transcript provided. Maintain confidenti

In [46]:
inputs = tokenizer(full_prompt, return_tensors="pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens=512)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Create a Medical SOAP note summary from the dialogue, following these guidelines:
S (Subjective): Summarize the patient's reported symptoms, including chief complaint and relevant history. Rely on the patient's statements as the primary source and ensure standardized terminology.
    O (Objective): Highlight critical findings such as vital signs, lab results, and imaging, emphasizing important details like the side of the body affected and specific dosages. Include normal ranges where relevant.
    A (Assessment): Offer a concise assessment combining subjective and objective data. State the primary diagnosis and any differential diagnoses, noting potential complications and the prognostic outlook.
    P (Plan): Outline the management plan, covering medication, diet, consultations, and education. Ensure to mention necessary referrals to other specialties and address compliance challenges.
    Considerations: Compile the report based solely on the transcript provided. Maintain confidenti